<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jcc/00-bow%2Bbinario%2Bsvm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline JCC: BoW+SVM

En esta notebook se presetan los experimentos sobre la estrategia de representación y técnica de aprendizaje *baseline* utilizada para las JCC de  la Universidad Nacional de La Plata.

Para ello vamos a preprocesar los correos y aplicar:
- Bag of words,
- Pesado binario/no binario,
- Máquina de vector soporte (SVM).


In [1]:
# Cargamos el archivo con las consultas que está en Github
from os import path

# En caso que no esté el archivo en Colab lo traigo
if not(path.exists('03-Correos_variables_estaticas.csv')):
  !wget https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/03-Correos_variables_estaticas.csv

In [2]:
# Leemos el archivo en un dataframe
import pandas as pd
df = pd.read_csv('03-Correos_variables_estaticas.csv', delimiter="|")

# Se transforma proveedor_correo a numerico
from sklearn import preprocessing
le_correo = preprocessing.LabelEncoder()
df['proveedor_correo'] = le_correo.fit_transform(df['proveedor_correo'])

In [3]:
# Me guardo los atributos, excepto la clase en x
x = df.drop(['Clase'], axis=1)
x.columns

Index(['Consulta', 'dia_semana', 'semana_del_mes', 'mes', 'cuatrimestre',
       'anio', 'hora_discretizada', 'dni_discretizado', 'legajo_discretizado',
       'posee_legajo', 'posee_telefono', 'carrera_valor', 'proveedor_correo',
       'cantidad_caracteres', 'proporcion_mayusculas', 'proporcion_letras',
       'cantidad_tildes', 'cantidad_palabras', 'cantidad_palabras_cortas',
       'proporcion_palabras_distintas', 'frecuencia_signos_puntuacion',
       'cantidad_oraciones', 'utiliza_codigo_asignatura'],
      dtype='object')

In [4]:
# Transformamos todas las Clases minoritarias (Puedo ir variando la cantidad de clases que derivo a la Clase "Otras Consultas")
cantidad_clases=4

clases = df.Clase.value_counts()
clases_minoritarias = clases.iloc[cantidad_clases:].keys().to_list()

df.Clase[df['Clase'].isin(clases_minoritarias)] = "Otras Consultas"

# Clases balanceadas
df.Clase.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Otras Consultas             330
Boleto Universitario        240
Ingreso a la Universidad    232
Requisitos de Ingreso       129
Pedido de Certificados       69
Name: Clase, dtype: int64

In [5]:
# Se numeriza la clase
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

# Me quedo con las clases numerizadas
y=le.fit_transform(df['Clase'])

# Por otro lado me guardo las etiquetas de las clases
target_names=le.classes_

In [6]:
# Separo datos de entrenamiento y testing
from sklearn.model_selection import train_test_split

# Separo en 80-20 entrenamiento/validación y testeo
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, test_size=0.2)

## SVM

Generamos el modelo:

In [7]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC

# Defino la feature y la transformación a aplicar en el texto
text_features = 'Consulta'
text_transformer = CountVectorizer()

# Defino la transformación
preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_transformer, text_features)
        ], remainder='passthrough')

# Combino la transformación con el pipeline
model_pipe = Pipeline([('preprocessor', preprocessor),
                       ('svm', SVC())])

# Defino los parámetros para GridSearchCV
parameters=[
        {'preprocessor__text__binary': [True, False],
         'preprocessor__text__analyzer': ['char'],
         'preprocessor__text__ngram_range': ((3, 3), (4, 7), (3, 4)),
         'preprocessor__text__strip_accents': ['unicode'],     
         'preprocessor__text__max_features':[500, 1000, 1500, 3000],
         'svm__C': [0.1, 1, 10, 100, 1000],  
         'svm__gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
         'svm__class_weight': [None, 'balanced'],
         'svm__kernel': ['rbf', 'linear', 'poly', 'rbf', 'sigmoid']
    }
]

# Instancio y "entreno" el GridSearchCV
grid_search=GridSearchCV(model_pipe, param_grid=parameters, cv=None, n_jobs=-1, verbose=3)
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   48.3s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('text',
                                                                         CountVectorizer(analyzer='word',
                                                                                         binary=False,
                                                                                         decode_error='strict',
                                                                                         dtype=<class 'numpy.int64'>,
                      

In [8]:
# Imprimo los mejores parámetros encontrados por GridSearchCV
print(grid_search.best_params_) 
  
# Imprimo el modelo después del ajuste de hiperparámetros
print(grid_search.best_estimator_) 

{'preprocessor__text__analyzer': 'char', 'preprocessor__text__binary': True, 'preprocessor__text__ngram_range': (3, 3)}
Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('text',
                                                  CountVectorizer(analyzer='char',
                                                                  binary=True,
                                                                  decode_error='strict',
                                                                  dtype=<class 'numpy.int64'>,
                                                                  encoding='utf-8',
                                                                  input='content',
                                                               

In [13]:
# Me autentico en Drive
from google.colab import drive
drive.mount('drive')

# Paso los resultados y el accuracy a un dataframe
results = pd.concat([pd.DataFrame(grid_search.cv_results_["params"]), pd.DataFrame(grid_search.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1)

# Los guardo en data_results.csv en mi Drive
results.to_csv('data_results.csv')
!cp data_results.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [10]:
from sklearn.metrics import classification_report, confusion_matrix 

# Se realizan las predicciones sobre el conjunto de validación
grid_predictions = grid_search.predict(x_test) 

# Se imprime el reporte de clasificación
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        47
           1       0.00      0.00      0.00        50
           2       0.33      1.00      0.50        66
           3       0.00      0.00      0.00        21
           4       0.00      0.00      0.00        16

    accuracy                           0.33       200
   macro avg       0.07      0.20      0.10       200
weighted avg       0.11      0.33      0.16       200



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Referencias
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- https://medium.com/analytics-vidhya/ml-pipelines-using-scikit-learn-and-gridsearchcv-fe605a7f9e05